In [1]:
from utils import *
from optimize import *
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import os
from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import train_test_split, KFold, RandomizedSearchCV
from sklearn.metrics import r2_score
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import OneHotEncoder

import xgboost as xgb

sns.set_style('ticks')

In [2]:
input_path = 'Data'

feature_file = 'deepnose_features.npy'
CID_file = 'molecules_train_cid.npy'
mixture_file = 'Mixure_Definitions_Training_set.csv'
training_task_file = 'TrainingData_mixturedist.csv'

features = np.load(os.path.join(input_path, feature_file))
mixtures_IDs = pd.read_csv(os.path.join(input_path, mixture_file))
training_set = pd.read_csv(os.path.join(input_path, training_task_file))

### Optimization for the RF and RGBoost regressors with  `X_feature`

In [3]:
feature_log_exp  = 0

In [4]:
if feature_log_exp: 
    scaler = StandardScaler(with_mean=True, with_std=True)
    features = scaler.fit_transform(features)
    features_CIDs = np.load(os.path.join(input_path, CID_file))
    CID2features =  {CID: features[i] for i, CID in enumerate(features_CIDs)}

    X, y, num_mixtures, all_pairs_CIDs = format_Xy(training_set,  mixtures_IDs, CID2features, method = 'log')

    
else:
    scaler = StandardScaler(with_mean=True, with_std=True)
    features = scaler.fit_transform(features)
    features_CIDs = np.load(os.path.join(input_path, CID_file))
    CID2features =  {CID: features[i] for i, CID in enumerate(features_CIDs)}

    X, y, num_mixtures, all_pairs_CIDs = format_Xy(training_set,  mixtures_IDs, CID2features, method = 'max')


In [5]:
# Convert the input pairs to a suitable format for training
X_pairs = np.array([(np.concatenate((x1, x2))) for x1, x2 in X])
y_true = np.array(y)

X_pair1 = X_pairs[:, :96] 
X_pair2 = X_pairs[:, 96:] 

In [6]:
# Embedding related summary features:
distances = [get_euclidean_distance(m[0], m[1]) for m in X]
similarities = [get_cosine_similarity(m[0], m[1]) for m in X]
angles = [get_cosine_angle(m[0], m[1]) for m in X]

In [7]:
# Mixture related summary features:
shared_monos = [ len( set(pair[0]).intersection(set(pair[1]))) for pair in all_pairs_CIDs]
diff_monos = [ len( set(pair[0]).difference(set(pair[1]))) for pair in all_pairs_CIDs]

In [8]:
# Dataset info
datasets = training_set['Dataset'].to_numpy()
encoder = OneHotEncoder()
data_arr = encoder.fit_transform(datasets.reshape(-1, 1))
data_arr = data_arr.toarray()

In [9]:
# Add features:
X_features = np.hstack((X_pairs, np.array(distances).reshape(500, 1), 
                        np.array(similarities).reshape(500, 1), 
                        np.array(angles).reshape(500, 1), 
                        np.array(shared_monos).reshape(500, 1), 
                        np.array(diff_monos).reshape(500, 1), 
                        np.array(num_mixtures), 
                        data_arr))

### Search for best parameter space

In [10]:
seeds = list(range(3))
rf_bests = []
rbg_bests = []
for seed in seeds:
    print(f"Random search for best hyperparams: round {seed +1} \n")
    rf_best,rbg_best = para_search(seed, X_features, y_true)
    rf_bests.append(rf_best)
    rbg_bests.append(rbg_best)

Random search for best hyperparams: round 1 



/Users/xinzheng/opt/anaconda3/envs/feedback/lib/python3.11/site-packages/sklearn/ensemble/_forest.py:413: FutureWarning: `max_features='auto'` has been deprecated in 1.1 and will be removed in 1.3. To keep the past behaviour, explicitly set `max_features=1.0` or remove this parameter as it is also the default value for RandomForestRegressors and ExtraTreesRegressors.
  warn(
/Users/xinzheng/opt/anaconda3/envs/feedback/lib/python3.11/site-packages/sklearn/ensemble/_forest.py:413: FutureWarning: `max_features='auto'` has been deprecated in 1.1 and will be removed in 1.3. To keep the past behaviour, explicitly set `max_features=1.0` or remove this parameter as it is also the default value for RandomForestRegressors and ExtraTreesRegressors.
  warn(
/Users/xinzheng/opt/anaconda3/envs/feedback/lib/python3.11/site-packages/sklearn/ensemble/_forest.py:413: FutureWarning: `max_features='auto'` has been deprecated in 1.1 and will be removed in 1.3. To keep the past behaviour, explicitly set `ma

Best Random Forest model:
Hyperparameters: {'n_estimators': 50, 'min_samples_split': 5, 'max_features': 'sqrt', 'max_depth': None, 'bootstrap': True}
Correlation: 0.9627628111070232
RMSE: 0.05502436548662802

Best XGBoost model:
Hyperparameters: {'subsample': 1.0, 'n_estimators': 200, 'max_depth': 7, 'learning_rate': 0.01, 'colsample_bytree': 0.5}
Correlation: 0.9869581113154721
RMSE: 0.047397279000637976
Random search for best hyperparams: round 2 



/Users/xinzheng/opt/anaconda3/envs/feedback/lib/python3.11/site-packages/sklearn/ensemble/_forest.py:413: FutureWarning: `max_features='auto'` has been deprecated in 1.1 and will be removed in 1.3. To keep the past behaviour, explicitly set `max_features=1.0` or remove this parameter as it is also the default value for RandomForestRegressors and ExtraTreesRegressors.
  warn(
/Users/xinzheng/opt/anaconda3/envs/feedback/lib/python3.11/site-packages/sklearn/ensemble/_forest.py:413: FutureWarning: `max_features='auto'` has been deprecated in 1.1 and will be removed in 1.3. To keep the past behaviour, explicitly set `max_features=1.0` or remove this parameter as it is also the default value for RandomForestRegressors and ExtraTreesRegressors.
  warn(
/Users/xinzheng/opt/anaconda3/envs/feedback/lib/python3.11/site-packages/sklearn/ensemble/_forest.py:413: FutureWarning: `max_features='auto'` has been deprecated in 1.1 and will be removed in 1.3. To keep the past behaviour, explicitly set `ma

Best Random Forest model:
Hyperparameters: {'n_estimators': 100, 'min_samples_split': 5, 'max_features': 'sqrt', 'max_depth': 10, 'bootstrap': True}
Correlation: 0.9463860288755971
RMSE: 0.06605264629753897

Best XGBoost model:
Hyperparameters: {'subsample': 1.0, 'n_estimators': 200, 'max_depth': 5, 'learning_rate': 0.01, 'colsample_bytree': 0.5}
Correlation: 0.9504899532511362
RMSE: 0.07013165965037192
Random search for best hyperparams: round 3 



/Users/xinzheng/opt/anaconda3/envs/feedback/lib/python3.11/site-packages/sklearn/ensemble/_forest.py:413: FutureWarning: `max_features='auto'` has been deprecated in 1.1 and will be removed in 1.3. To keep the past behaviour, explicitly set `max_features=1.0` or remove this parameter as it is also the default value for RandomForestRegressors and ExtraTreesRegressors.
  warn(
/Users/xinzheng/opt/anaconda3/envs/feedback/lib/python3.11/site-packages/sklearn/ensemble/_forest.py:413: FutureWarning: `max_features='auto'` has been deprecated in 1.1 and will be removed in 1.3. To keep the past behaviour, explicitly set `max_features=1.0` or remove this parameter as it is also the default value for RandomForestRegressors and ExtraTreesRegressors.
  warn(
/Users/xinzheng/opt/anaconda3/envs/feedback/lib/python3.11/site-packages/sklearn/ensemble/_forest.py:413: FutureWarning: `max_features='auto'` has been deprecated in 1.1 and will be removed in 1.3. To keep the past behaviour, explicitly set `ma

Best Random Forest model:
Hyperparameters: {'n_estimators': 50, 'min_samples_split': 2, 'max_features': 'sqrt', 'max_depth': 10, 'bootstrap': True}
Correlation: 0.956510217934419
RMSE: 0.062470896036352724

Best XGBoost model:
Hyperparameters: {'subsample': 1.0, 'n_estimators': 200, 'max_depth': 9, 'learning_rate': 0.01, 'colsample_bytree': 0.5}
Correlation: 0.9949297891778511
RMSE: 0.03701828557328093


### Get average performance for the best param combo 

In [13]:
rbg_bests

[{'subsample': 1.0,
  'n_estimators': 200,
  'max_depth': 7,
  'learning_rate': 0.01,
  'colsample_bytree': 0.5},
 {'subsample': 1.0,
  'n_estimators': 200,
  'max_depth': 5,
  'learning_rate': 0.01,
  'colsample_bytree': 0.5},
 {'subsample': 1.0,
  'n_estimators': 200,
  'max_depth': 9,
  'learning_rate': 0.01,
  'colsample_bytree': 0.5}]

In [14]:
rf_bests

[{'n_estimators': 50,
  'min_samples_split': 5,
  'max_features': 'sqrt',
  'max_depth': None,
  'bootstrap': True},
 {'n_estimators': 100,
  'min_samples_split': 5,
  'max_features': 'sqrt',
  'max_depth': 10,
  'bootstrap': True},
 {'n_estimators': 50,
  'min_samples_split': 2,
  'max_features': 'sqrt',
  'max_depth': 10,
  'bootstrap': True}]

In [35]:
rbg_best = rbg_bests[0]
rf_best = rf_bests[0]

In [36]:
# Random seeds to get average performance
random_seeds = [42, 123, 456, 789, 1011]
n_fold = 10  # Number of folds for cross-validation

xgb_corr_list = []
xgb_rmse_list = []

# Evaluate the models with different random seeds
for seed in random_seeds:
    np.random.seed(seed)
    
    # Create the XGBoost model with the best hyperparameters
    xgb_model = xgb.XGBRegressor(**rbg_best, random_state=seed)
    
    # Create the KFold object for cross-validation
    kf = KFold(n_splits=n_fold, shuffle=True, random_state=seed)
    
    xgb_corr_fold = []
    xgb_rmse_fold = []
    
    # Perform cross-validation
    for train_index, test_index in kf.split(X_features):
        X_train, X_test = X_features[train_index], X_features[test_index]
        y_train, y_test = y_true[train_index], y_true[test_index]
        
        # Train the model
        xgb_model.fit(X_train, y_train)
        
        # Evaluate the model on the testing fold
        xgb_pred = xgb_model.predict(X_test)
        xgb_corr = np.corrcoef(xgb_pred, y_test)[0, 1]
        xgb_rmse = np.sqrt(mean_squared_error(y_test, xgb_pred))
        
        xgb_corr_fold.append(xgb_corr)
        xgb_rmse_fold.append(xgb_rmse)
    
    # Calculate the average performance across all folds
    xgb_corr_avg = np.mean(xgb_corr_fold)
    xgb_rmse_avg = np.mean(xgb_rmse_fold)
    
    xgb_corr_list.append(xgb_corr_avg)
    xgb_rmse_list.append(xgb_rmse_avg)

In [37]:
print("XGBoost:")
print("R mean:", np.mean(xgb_corr_list))
print("R std:", np.std(xgb_corr_list))
print("RMSE mean:", np.mean(xgb_rmse_list))
print("RMSE std:", np.std(xgb_rmse_list))

XGBoost:
R mean: 0.6193319286146673
R std: 0.00685174491720304
RMSE mean: 0.12355518229568989
RMSE std: 0.00038598487638489453


In [38]:
# Random seeds to get average performance
random_seeds = [42, 123, 456, 789, 1011]
n_fold = 10  # Number of folds for cross-validation

rf_corr_list = []
rf_rmse_list = []

# Evaluate the models with different random seeds
for seed in random_seeds:
    np.random.seed(seed)
    
    # Create the XGBoost model with the best hyperparameters
    rf_model =  RandomForestRegressor(**rf_best, random_state=seed)
    
    # Create the KFold object for cross-validation
    kf = KFold(n_splits=n_fold, shuffle=True, random_state=seed)
    
    rf_corr_fold = []
    rf_rmse_fold = []
    
    # Perform cross-validation
    for train_index, test_index in kf.split(X_features):
        X_train, X_test = X_features[train_index], X_features[test_index]
        y_train, y_test = y_true[train_index], y_true[test_index]
        
        # Train the model
        rf_model.fit(X_train, y_train)
        
        # Evaluate the model on the testing fold
        rf_pred = rf_model.predict(X_test)
        rf_corr = np.corrcoef(rf_pred, y_test)[0, 1]
        rf_rmse = np.sqrt(mean_squared_error(y_test, rf_pred))
        
        rf_corr_fold.append(rf_corr)
        rf_rmse_fold.append(rf_rmse)
    
    # Calculate the average performance across all folds
    rf_corr_avg = np.mean(rf_corr_fold)
    rf_rmse_avg = np.mean(rf_rmse_fold)
    
    rf_corr_list.append(rf_corr_avg)
    rf_rmse_list.append(rf_rmse_avg)

In [39]:
print("RandomForest:")
print("R mean:", np.mean(rf_corr_list))
print("R std:", np.std(rf_corr_list))
print("RMSE mean:", np.mean(rf_rmse_list))
print("RMSE std:", np.std(rf_rmse_list))

RandomForest:
R mean: 0.5944725056504726
R std: 0.010848141551718406
RMSE mean: 0.12615760763131553
RMSE std: 0.0006135566872935029
